In [1]:
import os

import numpy as np
import virtualizarr as vz
import xarray as xr
from obstore.store import LocalStore
from virtualizarr.manifests.store import ObjectStoreRegistry

from hrrrparser import HRRRParser
from hrrrparser.codecs import LEVEL_COORDINATES

In [2]:
scheme = "file://"
prefix = os.getcwd()

In [3]:
parser = HRRRParser(steps=18)

In [4]:
file_urls = [
    f"{scheme}{prefix}/hrrr.t22z.wrfsfcf16.grib2",
    f"{scheme}{prefix}/hrrr.t23z.wrfsfcf16.grib2",
]
file_urls

['file:///Users/seanharkins/projects/hrrrparser/examples/hrrr.t22z.wrfsfcf16.grib2',
 'file:///Users/seanharkins/projects/hrrrparser/examples/hrrr.t23z.wrfsfcf16.grib2']

In [5]:
object_store = LocalStore()
registry = ObjectStoreRegistry({scheme: object_store})

In [6]:
manifest_store = parser(url=file_urls[1], registry=registry)

In [7]:
ms_ds = xr.open_dataset(
    manifest_store, engine="zarr", consolidated=False, zarr_format=3
)

In [8]:
ms_ds

<xarray.Dataset> Size: 57GB
Dimensions:                            (time: 1, step: 18, atm: 1, clt: 1,
                                        sfc: 1, hag: 8, isotherm: 2, isobar: 7,
                                        sigma: 1, entire_atm: 1,
                                        bndry_cloud: 1, lcl: 1, mcl: 1, hcl: 1,
                                        cld_ceiling: 1, clb: 1,
                                        zero_deg_isotherm: 1, htfl: 1,
                                        adiabatic_condensation_lifted: 1,
                                        eqm: 1, lfc: 1, y: 1059, x: 1799)
Coordinates: (12/23)
  * time                               (time) datetime64[s] 8B 2023-07-22T20:...
  * step                               (step) timedelta64[s] 144B 00:00:00 .....
  * atm                                (atm) float64 8B 0.0
  * clt                                (clt) float64 8B 0.0
  * sfc                                (sfc) float64 8B 0.0
  * hag                                (hag) float64 64B 0.0 1.0 ... 2e+03 4e+03
    ...                                 ...
  * htfl                               (htfl) float64 8B 0.0
  * adiabatic_condensation_lifted      (adiabatic_condensation_lifted) float64 8B ...
  * eqm                                (eqm) float64 8B 0.0
  * lfc                                (lfc) float64 8B 0.0
    latitude                           (y, x) float64 15MB ...
    longitude                          (y, x) float64 15MB ...
Dimensions without coordinates: y, x
Data variables: (12/69)
    refc_atm                           (time, step, atm, y, x) float64 274MB ...
    hail_atm                           (time, step, atm, y, x) float64 274MB ...
    ltng_atm                           (time, step, atm, y, x) float64 274MB ...
    tcdc_atm                           (time, step, atm, y, x) float64 274MB ...
    retop_clt                          (time, step, clt, y, x) float64 274MB ...
    pres_clt                           (time, step, clt, y, x) float64 274MB ...
    ...                                 ...
    hgt_htfl                           (time, step, htfl, y, x) float64 274MB ...
    rh_htfl                            (time, step, htfl, y, x) float64 274MB ...
    pres_htfl                          (time, step, htfl, y, x) float64 274MB ...
    hgt_adiabatic_condensation_lifted  (time, step, adiabatic_condensation_lifted, y, x) float64 274MB ...
    hgt_eqm                            (time, step, eqm, y, x) float64 274MB ...
    hgt_lfc                            (time, step, lfc, y, x) float64 274MB ...

In [9]:
ms_ds["tmp_isobar"].isel(y=100, x=100).values

array([[[  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,

In [10]:
loadable = LEVEL_COORDINATES + ["time", "step", "latitude", "longitude"]

vds1 = vz.open_virtual_dataset(
    url=file_urls[0],
    parser=parser,
    registry=registry,
    loadable_variables=loadable,
)
for var in loadable:
    if var in vds1:
        del vds1[var].encoding["serializer"]

In [11]:
vds2 = vz.open_virtual_dataset(
    url=file_urls[1],
    parser=parser,
    registry=registry,
    loadable_variables=loadable,
)
for var in loadable:
    if var in vds2:
        del vds2[var].encoding["serializer"]

In [12]:
import icechunk

storage = icechunk.in_memory_storage()
config = icechunk.RepositoryConfig.default()
container = icechunk.VirtualChunkContainer(
    url_prefix=f"{scheme}{prefix}/",
    store=icechunk.local_filesystem_store(f"{scheme}{prefix}/"),
)
config.set_virtual_chunk_container(container)
repo = icechunk.Repository.open_or_create(
    storage=storage,
    config=config,
    authorize_virtual_chunk_access={f"{scheme}{prefix}": None},
)
session = repo.writable_session("main")

In [13]:
encoding = vds1.time.encoding
encoding["units"] = "seconds since 1970-01-01"
encoding["calendar"] = "standard"
encoding["dtype"] = "int64"
vds1.time.encoding = encoding

In [14]:
vds1.vz.to_icechunk(session.store, validate_containers=False)

In [15]:
vds2.vz.to_icechunk(session.store, append_dim="time", validate_containers=False)

In [16]:
icy = xr.open_zarr(session.store, consolidated=False)
icy.time.encoding

{'chunks': (1,),
 'preferred_chunks': {'time': 1},
 'compressors': (),
 'filters': (),
 'shards': None,
 'serializer': BytesCodec(endian=<Endian.little: 'little'>),
 'units': 'seconds since 1970-01-01',
 'calendar': 'standard',
 'dtype': dtype('<i8')}

In [17]:
ds = xr.open_zarr(session.store, group="/", zarr_format=3, consolidated=False)
ds

<xarray.Dataset> Size: 113GB
Dimensions:                            (isotherm: 2, time: 2, step: 18, hag: 8,
                                        y: 1059, x: 1799, lfc: 1, htfl: 1,
                                        clt: 1,
                                        adiabatic_condensation_lifted: 1,
                                        clb: 1, mcl: 1, sfc: 1,
                                        zero_deg_isotherm: 1, entire_atm: 1,
                                        isobar: 7, atm: 1, eqm: 1,
                                        bndry_cloud: 1, hcl: 1, sigma: 1,
                                        cld_ceiling: 1, lcl: 1)
Coordinates: (12/23)
  * isotherm                           (isotherm) float64 16B 253.0 263.0
  * hag                                (hag) float64 64B 0.0 1.0 ... 2e+03 4e+03
  * lfc                                (lfc) float64 8B 0.0
    longitude                          (y, x) float64 15MB ...
  * clb                                (clb) float64 8B 0.0
  * eqm                                (eqm) float64 8B 0.0
    ...                                 ...
  * clt                                (clt) float64 8B 0.0
  * sigma                              (sigma) float64 8B 0.1
  * step                               (step) timedelta64[s] 144B 00:00:00 .....
  * isobar                             (isobar) float64 56B 2.5e+04 ... 1e+05
  * zero_deg_isotherm                  (zero_deg_isotherm) float64 8B 0.0
  * cld_ceiling                        (cld_ceiling) float64 8B 0.0
Dimensions without coordinates: y, x
Data variables: (12/69)
    tmp_hag                            (time, step, hag, y, x) float64 4GB ...
    rh_htfl                            (time, step, htfl, y, x) float64 549MB ...
    hgt_clt                            (time, step, clt, y, x) float64 549MB ...
    hgt_adiabatic_condensation_lifted  (time, step, adiabatic_condensation_lifted, y, x) float64 549MB ...
    maxuw_hag                          (time, step, hag, y, x) float64 4GB ...
    mcdc_mcl                           (time, step, mcl, y, x) float64 549MB ...
    ...                                 ...
    lcdc_lcl                           (time, step, lcl, y, x) float64 549MB ...
    hgt_isotherm                       (time, step, isotherm, y, x) float64 1GB ...
    wind_hag                           (time, step, hag, y, x) float64 4GB ...
    frozr_sfc                          (time, step, sfc, y, x) float64 549MB ...
    weasd_sfc                          (time, step, sfc, y, x) float64 549MB ...
    ltng_atm                           (time, step, atm, y, x) float64 549MB ...

In [18]:
ds["tmp_isobar"].isel(y=100, x=100).sel(
    time=np.datetime64("2025-07-10T19:00:00.000000000")
).values

array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.    

In [19]:
ds["tmp_isobar"].isel(y=100, x=100, step=16).values

array([[  0.        ,   0.        , 266.31001282, 283.92590332,
        295.52978516, 292.79974365, 290.33868408],
       [  0.        ,   0.        , 266.30953979, 285.00610352,
        296.15686035, 294.31420898, 291.6489563 ]])